### Importing libraries

In [7]:
from scipy.spatial import distance as dist
from imutils.video import VideoStream
from imutils import face_utils
import numpy as np
import argparse
import imutils
import dlib
import cv2
import keyboard
import morse_code  # This module should handle Morse code translation
import constants  # This module should define constants like EYE_AR_THRESH

### Eye Aspect Ratio (EAR)

The Eye Aspect Ratio (EAR) measures how the shape of the eye changes over time. It is useful for determining whether the eye is open or closed, which is important for applications such as detecting blinks or monitoring eye movement.

#### Formula
The formula for EAR is:

EAR = (A + B) / 2 x C

Where:
- **A** is the distance between the top and bottom vertical eye landmarks.
- **B** is the distance between the top and bottom vertical eye landmarks on the other side.
- **C** is the distance between the horizontal eye landmarks (the width of the eye).

#### Interpretation
- **EAR > Threshold**: The eye is considered open.
- **EAR < Threshold**: The eye is considered closed.

The threshold value is empirically determined and can vary based on individual characteristics. It is used to decide whether the eye is in an open or closed state.

In [8]:
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

### Face Detection and Eye Landmark Tracking Setup

This function sets up the tools and models needed to detect faces and track eye movements in real-time video.

1. **Load Face Detector and Landmark Predictor**:
   - It loads a pre-trained model to detect faces in video frames.
   - It also loads another model to find specific facial landmarks (points) around the eyes.

2. **Identify Eye Landmarks**:
   - It gets the indices (positions) of the facial landmarks specifically for the left and right eyes.

3. **Start Video Stream**:
   - It opens the webcam (or other video source) to start capturing live video frames.

4. **Return Setup Details**:
   - It returns everything needed for processing the video: the video stream object, face detector, landmark predictor, and the eye landmark indices.

#### Summary
In summary, this function sets up the tools and models required to detect faces and track eye movements in real-time video.

In [10]:
def setup_detector_video(args):
    print("[INFO] loading facial landmark predictor...")
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(args["shape_predictor"])
    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

    print("[INFO] starting video stream thread...")
    vs = VideoStream(src=0).start()
    return vs, detector, predictor, lStart, lEnd, rStart, rEnd

### Real-Time Morse Code Detection Using Eye Movements

This function performs real-time Morse code detection based on eye movements. Here’s a step-by-step overview of how it works:

1. **Initialize Counters and Flags**:
   - Sets up counters and flags to keep track of eye movements and Morse code timing.

2. **Capture Video Frames**:
   - Continuously captures frames from the camera.

3. **Detect Faces and Eyes**:
   - For each frame, detects faces and identifies the position of the eyes.

4. **Calculate Eye Aspect Ratio**:
   - Measures how closed the eyes are by calculating the distance between key points around the eyes.

5. **Draw Eye Contours**:
   - Highlights the eyes in the video feed for better visualization.

6. **Check Eye Status**:
   - If the eyes are closed for a certain amount of time, interprets this as a Morse code dot or dash.
   - If the eyes are open for a while, handles the separation between dots and dashes, or words.

7. **Update Morse Code**:
   - Updates and displays the current Morse code based on the eye movements.

8. **Display Information**:
   - Shows the current eye aspect ratio and Morse code on the screen.

9. **Handle User Input**:
   - Checks if a specific key is pressed to exit the program.

10. **Return Results**:
    - Returns the complete Morse code that was detected.

#### Summary
This process captures video frames, detects and analyzes eye movements to decode Morse code, and displays the results in real-time.


In [11]:
def loop_camera(vs, detector, predictor, lStart, lEnd, rStart, rEnd):
    # Initialize counters and flags
    COUNTER = 0               # Counts consecutive frames where eyes are closed
    BREAK_COUNTER = 0         # Counts frames where eyes are open to determine break periods
    EYES_OPEN_COUNTER = 0     # Counts consecutive frames where eyes are open
    CLOSED_EYES = False       # Flag to indicate if eyes are considered closed
    WORD_PAUSE = False        # Flag to indicate a pause between Morse code words
    PAUSED = False            # Flag to indicate if processing is paused

    # Initialize variables for storing Morse code results
    total_morse = ""          # Accumulates the entire translated Morse code
    morse_word = ""           # Stores the Morse code for the current word
    morse_char = ""           # Stores the Morse code for the current character

    while True:
        # Capture frame from video stream
        frame = vs.read()
        # Resize the frame for better processing
        frame = imutils.resize(frame, width=450)
        # Convert frame to grayscale for face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Detect faces in the frame
        rects = detector(gray, 0)

        for rect in rects:
            # Predict facial landmarks for each detected face
            shape = predictor(gray, rect)
            # Convert facial landmarks to numpy array
            shape = face_utils.shape_to_np(shape)
            # Extract left and right eye landmarks
            leftEye = shape[lStart:lEnd]
            rightEye = shape[rStart:rEnd]
            # Calculate eye aspect ratio for both eyes
            left_eye_ar = eye_aspect_ratio(leftEye)
            right_eye_ar = eye_aspect_ratio(rightEye)
            eye_ar = (left_eye_ar + right_eye_ar) / 2.0

            # Draw contours around the eyes
            leftEyeHull = cv2.convexHull(leftEye)
            rightEyeHull = cv2.convexHull(rightEye)
            cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

            # Check if the eye aspect ratio is below the threshold (indicating closed eyes)
            if eye_ar < constants.EYE_AR_THRESH:
                COUNTER += 1
                BREAK_COUNTER += 1
                # If eyes are closed for enough consecutive frames, mark as closed
                if COUNTER >= constants.EYE_AR_CONSEC_FRAMES:
                    CLOSED_EYES = True
                # If not paused, reset Morse character
                if not PAUSED:
                    morse_char = ""
                # If eyes have been closed for a long time, break out of the loop
                if BREAK_COUNTER >= constants.BREAK_LOOP_FRAMES:
                    break
            else:
                # Reset break counter if eyes are open
                if BREAK_COUNTER < constants.BREAK_LOOP_FRAMES:
                    BREAK_COUNTER = 0
                EYES_OPEN_COUNTER += 1
                # Add a dash to Morse code if eyes were closed long enough
                if COUNTER >= constants.EYE_AR_CONSEC_FRAMES:
                    morse_word += "-"
                    total_morse += "-"
                    morse_char += "-"
                    COUNTER = 0
                    CLOSED_EYES = False
                    PAUSED = True
                    EYES_OPEN_COUNTER = 0
                # Add a dot to Morse code if eyes were briefly closed
                elif CLOSED_EYES:
                    morse_word += "."
                    total_morse += "."
                    morse_char += "."
                    COUNTER = 1
                    CLOSED_EYES = False
                    PAUSED = True
                    EYES_OPEN_COUNTER = 0
                # Add a pause between Morse code characters if eyes are open long enough
                elif PAUSED and EYES_OPEN_COUNTER >= constants.PAUSE_CONSEC_FRAMES:
                    morse_word += "/"
                    total_morse += "/"
                    morse_char = "/"
                    PAUSED = False
                    WORD_PAUSE = True
                    CLOSED_EYES = False
                    EYES_OPEN_COUNTER = 0
                    # Write the current Morse code word to the keyboard
                    keyboard.write(morse_code.from_morse(morse_word))
                    morse_word = ""
                # Add a longer pause between words if eyes are open long enough
                elif WORD_PAUSE and EYES_OPEN_COUNTER >= constants.WORD_PAUSE_CONSEC_FRAMES:
                    total_morse += "¦/"
                    morse_char = ""
                    WORD_PAUSE = False
                    CLOSED_EYES = False
                    EYES_OPEN_COUNTER = 0
                    # Write a space to the keyboard to indicate a word separation
                    keyboard.write(morse_code.from_morse("¦/"))

            # Display the eye aspect ratio and Morse character on the frame
            cv2.putText(frame, "EAR: {:.2f}".format(eye_ar), (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, "{}".format(morse_char), (100, 200), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 2)

            # Print current Morse code word to the console
            print("\033[K", "morse_word: {}".format(morse_word), end="\r")

        # Show the frame in a window
        cv2.imshow("Frame", frame)
        # Check for user input to exit the loop
        key = cv2.waitKey(1) & 0xFF

        # Break the loop if the user presses the "]" key or if the break counter exceeds the threshold
        if key == ord("]") or BREAK_COUNTER >= constants.BREAK_LOOP_FRAMES:
            keyboard.write(morse_code.from_morse(morse_word))
            break

    # Return the total Morse code detected
    return total_morse

In [12]:
def cleanup(vs):
    cv2.destroyAllWindows()
    vs.stop()

In [13]:
def print_results(total_morse):
    print("Morse Code: ", total_morse.replace("¦", " "))
    print("Translated: ", morse_code.from_morse(total_morse))

In [14]:
def main():
    arg_par = argparse.ArgumentParser()
    arg_par.add_argument("-p", "--shape-predictor", required=True, help="path to facial landmark predictor")
    args = vars(arg_par.parse_args())

    (vs, detector, predictor, lStart, lEnd, rStart, rEnd) = setup_detector_video(args)
    total_morse = loop_camera(vs, detector, predictor, lStart, lEnd, rStart, rEnd)
    cleanup(vs)
    print_results(total_morse)

In [15]:
if __name__ == "__main__":
    main()

usage: ipykernel_launcher.py [-h] -p SHAPE_PREDICTOR
ipykernel_launcher.py: error: the following arguments are required: -p/--shape-predictor


SystemExit: 2

c:\Users\Hanon Systems\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
